In [20]:
import pandas as pd 
import re

data = pd.read_csv("data/standard_df.csv", sep=",")

In [21]:
from src.bias_mitigation import PreprocessingModel

In [22]:
model = PreprocessingModel()

In [23]:
model.load_and_split()
model.load_baseline()

c:\Users\dinat\Υπολογιστής\master\master thesis\fairness-ml\src\utils.py:28: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  return pd.read_csv(file_path, sep=sep)
c:\Users\dinat\Υπολογιστής\master\master thesis\fairness-ml\src\utils.py:28: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  return pd.read_csv(file_path, sep=sep)


In [24]:
from aif360.sklearn.preprocessing import LearnedFairRepresentations
from src.constants import LFR_PARAMS
from sklearn.metrics import make_scorer
from src.utils import average_odds_scoring, run_grid_search


lfr = LearnedFairRepresentations(
    model.protected_attrs[0],
    n_prototypes=25,
    max_iter=1000,
    random_state=0,
)

scorer_lfr = make_scorer(
    average_odds_scoring,
    scoring="delta",
    protected_attrs=model.protected_attrs[0],
)

lfr_grid = run_grid_search(
    lfr, LFR_PARAMS, model.X_train, model.y_train, scorer_lfr
)

y_pred_lfr = lfr_grid.predict(model.X_test)
y_proba_lfr = lfr_grid.predict_proba(model.X_test)[:, 1]

c:\Users\dinat\Υπολογιστής\master\master thesis\fairness-ml\.venv\lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
c:\Users\dinat\Υπολογιστής\master\master thesis\fairness-ml\.venv\lib\site-packages\aif360\sklearn\preprocessing\learning_fair_representations.py:163: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  warnings.warn('lbfgs failed to converge. Increase the number of '


In [ ]:
# In your test.ipynb, replace the last cell with:
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    f1_score,
    roc_auc_score,
)

from aif360.sklearn.metrics import (
    statistical_parity_difference,
    average_odds_difference,
    equal_opportunity_difference,
)

print(balanced_accuracy_score(model.y_test, y_pred_lfr))
print(f1_score(model.y_test, y_pred_lfr))
print(roc_auc_score(model.y_test, y_proba_lfr))

for attr in model.protected_attrs:
    print(f"\nFairness metrics for protected attribute: {attr}")
    
    # Pass the actual values from X_test, not just the column name
    spd = statistical_parity_difference(
        model.y_test,
        y_pred_lfr,
        prot_attr=model.X_test[attr]  # ✅ Pass the actual column values
    )
    aod = average_odds_difference(
        model.y_test,
        y_pred_lfr,
        prot_attr=model.X_test[attr],  # ✅ Pass the actual column values
    )
    eod = equal_opportunity_difference(
        model.y_test,
        y_pred_lfr,
        prot_attr=model.X_test[attr]  # ✅ Pass the actual column values
    )
    print(f"Statistical Parity Difference: {spd}")
    print(f"Average Odds Difference: {aod}")
    print(f"Equal Opportunity Difference: {eod}")

0.6934455859969559
0.7941381716678297
0.7663728225942837

Fairness metrics for protected attribute: Debtor
Statistical Parity Difference: 0.2681238812418986
Average Odds Difference: 0.0834908149798248
Equal Opportunity Difference: 0.06077044983542601

Fairness metrics for protected attribute: Gender
Statistical Parity Difference: 0.13224854310149092
Average Odds Difference: 0.050064183458177564
Equal Opportunity Difference: 0.0222928373371285

Fairness metrics for protected attribute: Scholarship holder
Statistical Parity Difference: -0.23287358380726342
Average Odds Difference: -0.1453792821477016
Equal Opportunity Difference: -0.099750015242973
